<a href="https://colab.research.google.com/github/Nocimaa/Binance_Bot/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv"

--2025-09-17 12:07:07--  https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28620838 (27M) [text/plain]
Saving to: ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’

btc_1h_adx_2017-08- 100%[===================>]  27.29M   172MB/s    in 0.2s    

2025-09-17 12:07:08 (172 MB/s) - ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’ saved [28620838/28620838]



In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("btc_1h_adx_2017-08-17_to_2025-09-17(in).csv")

df["label"] = df["open"].shift(-1)
df = df.drop("timestamp", axis=1)

df.dropna(inplace=True)

In [18]:
df.head()

,open,high,low,close,volume,ADX,+DI,-DI,ATR14,BB_width,...,MACD_signal,ROC,OBV,Vol_zscore,log_ret,close_open_ratio,high_low_spread,hour,dayofweek,label
0,3982.96,4055.92,3970.63,4043.63,49.206310,18.622436,12.743347,25.078866,62.194469,0.127883,...,-7.230313,-2.904721,-106.975164,0.339760,0.015118,1.015232,0.021092,9,0,4043.63
1,4043.63,4055.92,4010.78,4036.30,54.912335,19.621868,12.069528,23.752793,60.976292,0.128214,...,-10.137530,-2.526727,-161.887499,0.533848,-0.001814,0.998187,0.011184,10,0,4036.30
2,4036.30,4077.07,4035.82,4074.00,3.904708,19.893307,14.008617,22.577920,59.567272,0.126659,...,-12.037161,-1.831325,-157.982791,-2.139641,0.009297,1.009340,0.010125,11,0,4066.00
3,4066.00,4088.12,4066.00,4088.10,5.090504,19.814435,15.006875,21.950910,56.892466,0.125607,...,-12.953281,-1.963559,-152.892287,-2.133963,0.003455,1.005435,0.005411,12,0,4063.00
4,4063.00,4063.00,4000.01,4023.11,14.970767,20.869256,13.409755,27.587343,59.120862,0.127199,...,-14.212286,-1.546146,-167.863054,-1.530876,-0.016025,0.990182,0.015657,13,0,4023.11


In [19]:
y = df["label"]
X = df.drop("label", axis=1)

In [21]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

k = 10
selector = SelectKBest(score_func=f_regression, k=k)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [25]:
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # sortie unique = prédiction

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

model = RegressionNet(input_dim=k)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 30
batch_size = 32

In [ ]:
for epoch in range(epochs):
    permutation = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Évaluation train
    with torch.no_grad():
        preds = model(X_train_tensor)
        mse = criterion(preds, y_train_tensor).item()
    print(f"Epoch {epoch+1}/{epochs}, Train MSE={mse:.4f}")